
### Necessary packages 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [1]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [3]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [4]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")


11869883073013265539861

In [5]:
# Run this in case you want to import previous contract addresses
contracts_file_custom = "Updated_rewards _contracts_20210617233334.pkl"
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

## Helper Contracts and Variables

In [6]:
# Dont run this in case you have imported contracts from a pkl file 
contracts = {}
PREP_LIST = ["hxec79e9c1c882632688f8c8f9a07832bcabe8be8f","hxd3be921dfe193cd49ed7494a53743044e3376cd3",    "hx9e7509f86ea3ba5c139161d6e92a3982659e9f30","hxaad52424d4aec9dac7d9f6796da527f471269d2c"]
TIMESTAMP = 1622560500000000
EMISSION_PER_ASSET = (400000 * 10 ** 18 ) // (6 * 86400)
contracts['sicx']="cxae6334850f13dfd8b50f8544d5acb126bb8ef82d"
contracts['staking']="cx9d829396d887f9292d8af488fab78ad24ab6b99a"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
contracts['usds']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"
contracts['bandOracle'] = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
contracts["dex"] = "cx399dea56cf199b1c9e43bead0f6a284bdecfbf62"


In [42]:
TIMESTAMP = 1622560500000000
EMISSION_PER_ASSET = (400000 * 10 ** 18 ) // (6 * 86400)

# Deploying all contracts


In [7]:

deploy = ['addressProvider','daoFund','delegation','feeProvider','governance', 
          'lendingPool', 'lendingPoolCore','lendingPoolDataProvider',
          'liquidationManager','ommToken', 'priceOracle','rewardDistribution' ,
          'snapshot','worker_token']

directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
        
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xadfb7a029004f350fe771d4ce19a093c7aea80e044ab9b00afac7758f737cf27', 'blockHeight': 18231711, 'blockHash': '0x63029665ca126aa8266d055c28bf61d9c17acf4b1c0d006a5611c87c5d80b097', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9', 'stepUsed': 1040146400, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1040146400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x008a05f809de4605332d2cd32ee33db74bd2547d5c0f97454ee785ea4abd36d5', 'blockHeight': 18231745, 'blockHash': '0xf60167873138592b69ddbbd93b24ab6b85b93bb22f560c306f7d8e595a38910a', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx26997b6c4fecae3badf393450056ce06c66227fd', 'stepUsed': 1067202400, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1067202400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xf282bdedfef0e0924c5fcb9a0d181a4c93b9466553cbd60083112a830d313f21', 'blockHeight': 18231748, 'blockHash': '0x96d18f798bb3ed28d38a7d087511c77e322ebf86e8bceafc065ecc910603ceb9', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx17d55d3f5e3bc8542c34fe115e42c50a9ab71bec', 'stepUsed': 1050195280, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1050195280, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [9]:
pprint(contracts)

{'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dICX': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846',
 'dIUSDC': 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b',
 'dUSDS': 'cx242349f8c338977dba062d728f2627831ba11cd9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'oICX': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a',
 'oIUSDC': 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa',
 'oUSDS': 'cx97fd33b6002f0bb0a20

## Deploying remaining contracts


In [ ]:
for item in deploy:
    if item not in contracts.keys():
        params = {}
        if item == "ommToken":
            params = {'_initialSupply':0, '_decimals': 18}
        elif item == "worker_token":
            params = {'_initialSupply':100, '_decimals': 18}
       
        deploy_transaction = DeployTransactionBuilder()\
            .from_(deployer_wallet.get_address())\
            .to(GOVERNANCE_ADDRESS)\
            .nid(NID)\
            .nonce(100)\
            .content_type("application/zip")\
            .content(gen_deploy_data_content(item))\
            .params(params)\
            .build()

        step_limit = icon_service.estimate_step(deploy_transaction) + 100000
        
        signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
        tx_hash = icon_service.send_transaction(signed_transaction)

        res = get_tx_result(tx_hash)
        print(res)
        contracts[item] = res.get('scoreAddress', '')

## Deploying otoken for USDS

In [21]:

params = {"_name":"Omm USDS Interest Token","_symbol":"oUSDS"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDS'] = res.get('scoreAddress', '')



contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x7ca0cf3922744afa1b62b2419630b7fd23d3d1a11d45a915a92d449124e8a43e', 'blockHeight': 18232061, 'blockHash': '0xf15512fa66360e61a0cd1d63e14088e24565ac833f22fccbbd1e0ddd1c336397', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx97fd33b6002f0bb0a20f8300e2a10f89a2a390e4', 'stepUsed': 1172743000, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1172743000, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'staking': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'usds': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'ommToken': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
 'priceOracle': 'cxc6d71d4db1c09fd8d2ec1892aea56b462abd1110',
 'rewardDistribution': 

## Deploying debt token for usds (dUSDS)

In [10]:
params = {"_name":"Omm USDS Debt Token","_symbol":"dUSDS"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('dToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['dUSDS'] = res.get('scoreAddress', '')



contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x052214c3cb57e3abef094c01315b2537e148f6947b5528cf328ce1cfd13223a8', 'blockHeight': 18231798, 'blockHash': '0x9d8dc2376afe31ba7c89793f1e3eb011c55a58bd986da63c2d46bb05ee459c6f', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx242349f8c338977dba062d728f2627831ba11cd9', 'stepUsed': 1147630520, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1147630520, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'staking': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'usds': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'ommToken': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
 'priceOracle': 'cxc6d71d4db1c09fd8d2ec1892aea56b462abd1110',
 'rewardDistribution': 

## Deploying otoken for icx

In [11]:
#deploy oICX 
params = {"_name":"Omm ICX interest Token","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xea7a48bf0802e2ed3eedcaeda53b3fad0979665ac6f39856d89852a536bbaaf2', 'blockHeight': 18231802, 'blockHash': '0x5c476c03c6d819f2e0afcbf6e18a49ac19a24e228066aca52df085b2fd3217a8', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a', 'stepUsed': 1172741960, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1172741960, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'staking': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'usds': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'ommToken': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
 'priceOracle': 'cxc6d71d4db1c09fd8d2ec1892aea56b462abd1110',
 'rewardDistribution': 

## Deploying debt token for icx (dICX)

In [12]:

params = {"_name":"Omm ICX Debt Token","_symbol":"dICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('dToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['dICX'] = res.get('scoreAddress', '')



contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x3b46db299f52ad8b2985f3c4a126b7d339d0f726a6194bed4823e8efef717638', 'blockHeight': 18231807, 'blockHash': '0xeb684fe582c2ea94b40a9b6584ff7ec111375e668f53be47654db6f951fa7651', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846', 'stepUsed': 1147629480, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1147629480, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'staking': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'usds': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'ommToken': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
 'priceOracle': 'cxc6d71d4db1c09fd8d2ec1892aea56b462abd1110',
 'rewardDistribution': 

## Deploying otoken for iusdc


In [13]:
#deploy oIUSDC
params = {"_name":"Omm IUSDC Interest Token","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x8f2e7cda768fa0212bbb9a53f4709271d54d362242dc6b407e48a6b431ae917a', 'blockHeight': 18231811, 'blockHash': '0xa18cf0747c159b2aa03d1d962fa7651902c2ed61c9774edc767025d08f3a4f23', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa', 'stepUsed': 1172747640, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1172747640, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'staking': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'usds': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'ommToken': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
 'priceOracle': 'cxc6d71d4db1c09fd8d2ec1892aea56b462abd1110',
 'rewardDistribution': 

## Deploying debt token for iusdc dIUSDC

In [14]:

params = {"_name":"Omm IUSDC Debt Token","_symbol":"dIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('dToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['dIUSDC'] = res.get('scoreAddress', '')



contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x5d6347865dfbff69bb21cfb156eb466e80fe3305121a767f1da39e0997328670', 'blockHeight': 18231816, 'blockHash': '0xff1c63006092011b517e806f4552d679f021994c6b4599917911d725ec748e5a', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b', 'stepUsed': 1147635160, 'stepPrice': 12500000000, 'cumulativeStepUsed': 1147635160, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'sicx': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
 'staking': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'usds': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'ommToken': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
 'priceOracle': 'cxc6d71d4db1c09fd8d2ec1892aea56b462abd1110',
 'rewardDistribution': 

## Other Contract Addresses

In [7]:
pprint(contracts)

{'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dICX': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846',
 'dIUSDC': 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b',
 'dUSDS': 'cx242349f8c338977dba062d728f2627831ba11cd9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'oICX': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a',
 'oIUSDC': 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa',
 'oUSDS': 'cx97fd33b6002f0bb0a20

## Saving the pkl file


In [23]:
now = datetime.utcnow()
filename ="Updated_rewards "+f'_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'

print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

saving contracts list as Updated_rewards _contracts_20210617233334.pkl


## Updating contracts

In [119]:
# Updating mainnet contracts
contract_ = 'rewardDistribution'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDS':
    update = contracts['oUSDS']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

update rewardDistribution cx791e6684e6861ba1add73b5cf402080059ae9cee


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xd47f2ddeac17017a386987e95c2396fbd782b7ec358b0ab532f4c55fbb7ecee9',
 'blockHeight': 18253389,
 'blockHash': '0x10190ae499e5e79c9e3a25156d00178ad76ac5375fee346847a1de2431b931e6',
 'txIndex': 1,
 'to': 'cx791e6684e6861ba1add73b5cf402080059ae9cee',
 'scoreAddress': 'cx791e6684e6861ba1add73b5cf402080059ae9cee',
 'stepUsed': 1790504800,
 'stepPrice': 12500000000,
 'cumulativeStepUsed': 1790504800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Configuring SCOREs's parameters

In [36]:
settings_lendingPool = [
    {'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
    {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
    {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
    {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
    {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
    {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
    {'contract': 'lendingPool', 'method': 'setRewardManager','params': {'_address':contracts['rewardDistribution']}},
    {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
    {'contract': 'lendingPool', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
    {'contract': 'lendingPool', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}}
]
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))



Calling setRewardManager, with parameters {'_address': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the lendingPool contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xa1724aaffaa7ac8500ab47b057ddd9e757081530060707fb50962603cb1db6d2', 'blockHeight': 18251243, 'blockHash': '0xe76b63aa2b5a59c0c915321211480dcd0ad73ad01b887de6bd2f3ea50b2ccf2a', 'txIndex': 1, 'to': 'cx16c14999d0bf62f3875343f27c10efe4ce107623', 'stepUsed': 150920, 'stepPrice': 12500000000, 'cumulativeStepUsed': 150920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [ ]:
settings_feeProvider = [
    {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
    
]
for sett in settings_feeProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))

In [19]:
settings_lendinPoolDataProvider=[
    {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['usds'],'_sym':"USDS"}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
    {'contract': 'lendingPoolDataProvider', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}}
]
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setSymbol, with parameters {'_reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_sym': 'USDS'} on the lendingPoolDataProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setSymbol, with parameters {'_reserve': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', '_sym': 'ICX'} on the lendingPoolDataProvider contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setStaking, with parameters {'_address': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a'} on the lendingPoolDataProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [24]:
settings_oToken =[
    {'contract': 'oUSDS', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'oUSDS', 'method': 'setReserve', 'params':{'_address':contracts['usds']}},
    {'contract': 'oUSDS', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'oUSDS', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'oUSDS', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},
    {'contract': 'oUSDS', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the oUSDS contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'} on the oUSDS contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDistributionManager, with parameters {'_address': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the oUSDS contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [25]:
setting_priceOracle =[
    {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
    {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
    {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'USDC','_rate':10*10**17}}
]
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling set_reference_data, with parameters {'_base': 'USDb', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling set_reference_data, with parameters {'_base': 'ICX', '_quote': 'USD', '_rate': 1000000000000000000} on the priceOracle contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling set_reference_data, with parameters {'_base': 'IUSDC', '_quote': 'USDC', '_rate': 1000000000000000000} on the priceOracle contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [26]:
setting_addressProvider =[  
    {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'addressProvider', 'method': 'setUSDs', 'params':{'_address':contracts['usds']}},
    {'contract': 'addressProvider', 'method': 'setoUSDs', 'params':{'_address':contracts['oUSDS']}},
    {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
    {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
    {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
    {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
    {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
    {'contract': 'addressProvider', 'method': 'setOmmToken', 'params':{'_address':contracts['ommToken']}},
    {'contract': 'addressProvider', 'method': 'setDelegation', 'params':{'_address':contracts['delegation']}},
    {'contract': 'addressProvider', 'method': 'setRewards', 'params':{'_address':contracts['rewardDistribution']}},
    {'contract': 'addressProvider', 'method': 'setdUSDs', 'params':{'_address':contracts['dUSDS']}},
    {'contract': 'addressProvider', 'method': 'setdIUSDC', 'params':{'_address':contracts['dIUSDC']}},
    {'contract': 'addressProvider', 'method': 'setdICX', 'params':{'_address':contracts['dICX']}}
]

failed_methods = []

for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])
    if get_tx_result(tx_hash)['status'] != 1:
        failed_methods.append(sett["method"])
print(failed_methods)



Calling setLendingPool, with parameters {'_address': 'cx16c14999d0bf62f3875343f27c10efe4ce107623'} on the addressProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolDataProvider, with parameters {'_address': 'cx29d158fac7a47a246a15888e123396f25b580ad8'} on the addressProvider contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setdICX, with parameters {'_address': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846'} on the addressProvider contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [27]:
settings_reserves = [{'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['usds'],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
                    {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['iusdc'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}}
                    ]
for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '20000000000000000', 'slopeRate1': '60000000000000000', 'slopeRate2': '1000000000000000000'}]} on the lendingPoolCore contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '000000000000000000', 'slopeRate1': '80000000000000000', 'slopeRate2': '2000000000000000000'}]} on the lendingPoolCore contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserveConstants, with parameters {'_constants': [{'reserve': 'cx65f639254090820361da483df233f6d0e69af9b7', 'optimalUtilizationRate': '800000000000000000', 'baseBorrowRate': '20000000000000000', 'slopeRate1': '60000000000000000', 'slopeRate2': '1000000000000000000'}]} on the lendingPoolCore contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [28]:
settings_oicx = [
    {'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
    {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'oICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},
    {'contract': 'oICX', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]

failed_methods = []

for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the oICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d'} on the oICX contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDistributionManager, with parameters {'_address': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the oICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [31]:
settings_oiusdc = [ 
    {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'oIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},    
    {'contract': 'oIUSDC', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}}
]
failed_methods = []
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setDistributionManager, with parameters {'_address': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the oIUSDC contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1
[]


In [32]:
pprint(contracts)

{'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dICX': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846',
 'dIUSDC': 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b',
 'dUSDS': 'cx242349f8c338977dba062d728f2627831ba11cd9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'oICX': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a',
 'oIUSDC': 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa',
 'oUSDS': 'cx97fd33b6002f0bb0a20

In [33]:
# feeProvider is daoFund for now
settings_liquidationManager =  [
    {'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
    {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
    {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}},    
    {'contract': 'liquidationManager', 'method': 'setStaking','params': {'_address':contracts['staking']}}
]
failed_methods = []
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setLendingPoolDataProvider, with parameters {'_address': 'cx29d158fac7a47a246a15888e123396f25b580ad8'} on the liquidationManager contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the liquidationManager contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setStaking, with parameters {'_address': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a'} on the liquidationManager contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1
[]


In [34]:
settings_delegation =[
    {'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
    {'contract': 'delegation', 'method': 'addAllContributors','params': {'_preps':PREP_LIST}},
    {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}},    
#     {'contract':  'delegation', 'method': 'setVoteThreshold','params': {'_vote':    }}
]
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the delegation contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling addAllContributors, with parameters {'_preps': ['hxec79e9c1c882632688f8c8f9a07832bcabe8be8f', 'hxd3be921dfe193cd49ed7494a53743044e3376cd3', 'hx9e7509f86ea3ba5c139161d6e92a3982659e9f30', 'hxaad52424d4aec9dac7d9f6796da527f471269d2c']} on the delegation contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setOmmToken, with parameters {'_address': 'cx0bc91748db9325167f19eeef13b918c76d3768e2'} on the delegation contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [35]:
settings_ommToken =[
    {'contract':  'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewardDistribution']}},
    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewardDistribution']}},
    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':10 * 10**18}},
    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}
]
failed_methods = []
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setAdmin, with parameters {'_admin': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the ommToken contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDelegation, with parameters {'_address': 'cx15f02047a6def31b190af9e126029af5a5dac428'} on the ommToken contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setUnstakingPeriod, with parameters {'_time': 120} on the ommToken contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1
[]


In [36]:
settings_lendingPoolCore =[ 
    {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
    {'contract': 'lendingPoolCore', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
#     {'contract': 'lendingPoolCore', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
    {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
    {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
    {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
    {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}}
]
failed_methods = []
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setLendingPool, with parameters {'_address': 'cx16c14999d0bf62f3875343f27c10efe4ce107623'} on the lendingPoolCore contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDaoFund, with parameters {'_address': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997'} on the lendingPoolCore contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling set_id, with parameters {'_value': '1'} on the lendingPoolCore contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1
[]


In [37]:
settings_priceOracle =[
    {'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':contracts['bandOracle']}},
    {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}
]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setBandOracle, with parameters {'_address': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'} on the priceOracle contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling toggleOraclePriceBool, with parameters {} on the priceOracle contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [29]:
pprint(contracts)

{'addressProvider': 'cxe3fa5aa4403f25c1b9cf53a38fd73dcce099bab9',
 'bandOracle': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964',
 'dICX': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846',
 'dIUSDC': 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b',
 'dUSDS': 'cx242349f8c338977dba062d728f2627831ba11cd9',
 'daoFund': 'cxd4570ce5676f74f251095f94a557b7d15ad1b997',
 'delegation': 'cx15f02047a6def31b190af9e126029af5a5dac428',
 'dex': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62',
 'governance': 'cx2648fcffffbe693a7017b39d5220f71ada06577e',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'lendingPoolCore': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'lendingPoolDataProvider': 'cx29d158fac7a47a246a15888e123396f25b580ad8',
 'liquidationManager': 'cxf3f0038def60f8bd9bc1f26c4ff8c8a524c37647',
 'oICX': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a',
 'oIUSDC': 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa',
 'oUSDS': 'cx97fd33b6002f0bb0a20

In [39]:
settings_snapshot =[
    {'contract': 'snapshot', 'method': 'setAdmin','params': {'_address':contracts['lendingPool']}},
    {'contract': 'snapshot', 'method': 'setGovernance','params': {'_address':contracts['governance']}}
]
for sett in settings_snapshot:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setAdmin, with parameters {'_address': 'cx16c14999d0bf62f3875343f27c10efe4ce107623'} on the snapshot contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setGovernance, with parameters {'_address': 'cx2648fcffffbe693a7017b39d5220f71ada06577e'} on the snapshot contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [6]:
# REWARDS CONTRACT NOT IN USE


# def setDistPercentage(self, _deposit: int, _borrow: int, _ommICX: int, _ommUSDb: int, _worker: int, _daoFund: int):
dist_params = { 
    "_ommICX": 1*10**17, 
    "_dex": 1*10**17, 
    "_worker": 2*10**17, 
    "_daoFund": 2*10**17}
setting_reward_percentage =[{'contract':'rewardDistribution','method':'setDistPercentage','params':dist_params}]
for sett in setting_reward_percentage:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))




Calling setDistPercentage, with parameters {'_ommICX': 100000000000000000, '_dex': 100000000000000000, '_worker': 200000000000000000, '_daoFund': 200000000000000000} on the rewardDistribution contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xbc7a224adb04ddcdbc1716c1a9c8c7931129d2cbb6f60101ba6e9d3af51b7588', 'blockHeight': 18249031, 'blockHash': '0x3072280f2806cd94ca09cbc6a196d14cb672618552b67e903d207336da8f47eb', 'txIndex': 1, 'to': 'cx791e6684e6861ba1add73b5cf402080059ae9cee', 'stepUsed': 159360, 'stepPrice': 12500000000, 'cumulativeStepUsed': 159360, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [43]:
settings_rewards= settings_rewards = [
#             {'contract': 'rewardDistribution', 'method': 'setLendingPoolDataProvider',
#              'params': {'_address': contracts['lendingPoolDataProvider']}},
#             {'contract': 'rewardDistribution', 'method': 'setOmm',
#              'params': {'_address': contracts['ommToken']}},
            {'contract': 'rewardDistribution', 'method': 'configureAssetEmission',
             'params': {
                "_assetConfig": 
                    [
                        {"asset": contracts["oUSDS"], "emissionPerSecond":f"{EMISSION_PER_ASSET}",'totalBalance': "0" },
                        {"asset": contracts["dUSDS"], "emissionPerSecond":f"{EMISSION_PER_ASSET}",'totalBalance': "0" },
                        {"asset": contracts["dICX"], "emissionPerSecond":f"{EMISSION_PER_ASSET}",'totalBalance': "0" },
                        {"asset": contracts["oICX"], "emissionPerSecond":f"{EMISSION_PER_ASSET}",'totalBalance': "0"}, 
                        {"asset": contracts["oIUSDC"], "emissionPerSecond":f"{EMISSION_PER_ASSET}",'totalBalance': "0" },
                        {"asset": contracts["dIUSDC"], "emissionPerSecond":f"{EMISSION_PER_ASSET}",'totalBalance': "0"} 
                    ]             
                }
            },
#             {'contract': 'rewardDistribution', 'method': 'setWorkerToken',
#              'params': {'_address': contracts['worker_token']}},
#             {'contract': 'rewardDistribution', 'method': 'setAdmin',
#              'params': {'_address': contracts['governance']}},
#             {'contract': 'rewardDistribution', 'method': 'setDaoFund',
#              'params': {'_address': contracts['daoFund']}},
#             {'contract': 'rewardDistribution', 'method': 'setLpToken',
#              'params': {'_address': contracts['dex']}}
]

for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling configureAssetEmission, with parameters {'_assetConfig': [{'asset': 'cx97fd33b6002f0bb0a20f8300e2a10f89a2a390e4', 'emissionPerSecond': '771604938271604938', 'totalBalance': '0'}, {'asset': 'cx242349f8c338977dba062d728f2627831ba11cd9', 'emissionPerSecond': '771604938271604938', 'totalBalance': '0'}, {'asset': 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846', 'emissionPerSecond': '771604938271604938', 'totalBalance': '0'}, {'asset': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a', 'emissionPerSecond': '771604938271604938', 'totalBalance': '0'}, {'asset': 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa', 'emissionPerSecond': '771604938271604938', 'totalBalance': '0'}, {'asset': 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b', 'emissionPerSecond': '771604938271604938', 'totalBalance': '0'}]} on the rewardDistribution contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


In [44]:
tx_hash

'0xb1b22c3afb60e25aa0acd1ed302ca01dc155d1dd2689ecf3fc4972a588366993'

## Timestamp to be added 

In [16]:
settings_governance =[
    {'contract': 'governance', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
    {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewardDistribution']}},
    {'contract': 'governance', 'method': 'setStartTimestamp','params': {'_timestamp': TIMESTAMP}}, # REWARDS ma we need setStartTimestamp
]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSnapshot, with parameters {'_address': 'cx26997b6c4fecae3badf393450056ce06c66227fd'} on the governance contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x3f7a547ccc5ca346c2418a86b5fe02b1383343f7b8e354182c2198add2251820', 'blockHeight': 18242468, 'blockHash': '0xb7b5b596967658976dce99e49ace843bd3e32c4c5d4e4c1879719385c8efc160', 'txIndex': 1, 'to': 'cx2648fcffffbe693a7017b39d5220f71ada06577e', 'stepUsed': 144880, 'stepPrice': 12500000000, 'cumulativeStepUsed': 144880, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x7944ff89c0dfbe50e6ecfde090afc39f646caa5a83e97a96056f9c8528d0ec85', 'blockHeight': 18242470, 'blockHash': '0xed115a05599b53d207781bfc58f3b32326d2e7e7cbd932f581d8fbefe4c135e2', 'txIndex': 1, 'to': 'cx2648fcffffbe693a7017b39d5220f71ada06577e', 'stepUsed': 144680, 'stepPrice': 12500000000, 'cumulativeStepUsed': 144680, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x307b068cd66c5ad1d22de6bf479732ada095aa9a129d2f9e4ded670148c9a2c5', 'blockHeight': 18242473, 'blockHash': '0xe1422eb448a8aa07261e83b52f6ff2933323e056138e6e9a8ae3267bc2bfc266', 'txIndex': 1, 'to': 'cx2648fcffffbe693a7017b39d5220f71ada06577e', 'stepUsed': 193880, 'stepPrice': 12500000000, 'cumulativeStepUsed': 193880, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Configure ICX Debt Token

In [27]:
settings_dicx = [
    {'contract': 'dICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'dICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
    {'contract': 'dICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'dICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'dICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},
    {'contract': 'dICX', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]

failed_methods = []
for sett in settings_dicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the dICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d'} on the dICX contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDistributionManager, with parameters {'_address': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the dICX contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1
[]


## Configure USDS Debt Token

In [43]:
settings_dUSDS = [
    {'contract': 'dUSDS', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'dUSDS', 'method': 'setReserve', 'params':{'_address':contracts['usds']}},
    {'contract': 'dUSDS', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'dUSDS', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'dUSDS', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},    
    {'contract': 'dUSDS', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]
failed_methods = []
for sett in settings_dUSDS:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)



Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the dUSDS contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'} on the dUSDS contract.


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 8 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 16 seconds...


ReadTimeout: HTTPSConnectionPool(host='bicon.net.solidwallet.io', port=443): Read timed out. (read timeout=10)

## Configure IUSDC Debt Token

In [44]:
settings_diusdc =[
    {'contract': 'dIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
    {'contract': 'dIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
    {'contract': 'dIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
    {'contract': 'dIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
    {'contract': 'dIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}},  
    {'contract': 'dIUSDC', 'method': 'setDistributionManager', 'params':{'_address':contracts['rewardDistribution']}},
]

failed_methods = []

for sett in settings_diusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    status = get_tx_result(tx_hash)['status']
    print(status)
    if status != 1:
        failed_methods.append(sett['method'])
print(failed_methods)
    



Calling setLendingPoolCore, with parameters {'_address': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757'} on the dIUSDC contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setReserve, with parameters {'_address': 'cx65f639254090820361da483df233f6d0e69af9b7'} on the dIUSDC contract.


...

{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1


Calling setDistributionManager, with parameters {'_address': 'cx791e6684e6861ba1add73b5cf402080059ae9cee'} on the dIUSDC contract.


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
1
[]


## Adding USDS reserve in LendingPoolCore



In [45]:
params ={
    "_reserve": {
        "reserveAddress":contracts['usds'],
        "oTokenAddress":contracts['oUSDS'],
        "dTokenAddress": contracts['dUSDS'],
        "lastUpdateTimestamp": "0",
        "liquidityRate":"0",
        "borrowRate":"0",
        "liquidityCumulativeIndex":f"1{'0'*18}",
        "borrowCumulativeIndex":f"1{'0'*18}",
        "baseLTVasCollateral":"500000000000000000",
        "liquidationThreshold":"650000000000000000",
        "liquidationBonus":"100000000000000000",
        "decimals":"18",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled":"1",
        "isFreezed":"0",
        "isActive":"1"
    } 
}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x6e869739f0dcdb1b27774165c6f1a0efa56466b1cbccdee3eb8c9423ee59b269',
 'blockHeight': 18233051,
 'blockHash': '0xcbf9d5666fc7fbe9349236e6e64e3e3e84745248116609502ef524f56df56f8f',
 'txIndex': 1,
 'to': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'stepUsed': 291760,
 'stepPrice': 12500000000,
 'cumulativeStepUsed': 291760,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding ICX reserve

In [46]:
params ={
    "_reserve": {
        "reserveAddress":contracts['sicx'],
        "oTokenAddress":contracts['oICX'],
        "dTokenAddress": contracts['dICX'],
        "lastUpdateTimestamp": "0",
        "liquidityRate":"0",
        "borrowRate":"0",
        "liquidityCumulativeIndex":f"1{'0'*18}",
        "borrowCumulativeIndex":f"1{'0'*18}",
        "baseLTVasCollateral":"500000000000000000",
        "liquidationThreshold":"650000000000000000",
        "liquidationBonus":"100000000000000000",
        "decimals":"18",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled":"1",
        "isFreezed":"0",
        "isActive":"1"
    } 
}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x9aeeaa2087d8284097ab39f4cffd5f041fdb279ad1b9ef956c28241fefbdf1f4',
 'blockHeight': 18233056,
 'blockHash': '0x81fc3b3c5023dfd9a33614ed169ea0e0aca1e8f9fa1a69f00e0995aa57e3b741',
 'txIndex': 1,
 'to': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'stepUsed': 291520,
 'stepPrice': 12500000000,
 'cumulativeStepUsed': 291520,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding IUSDC reserve

In [47]:
params ={
    "_reserve": {
        "reserveAddress":contracts['iusdc'],
        "oTokenAddress":contracts['oIUSDC'],
        "dTokenAddress": contracts['dIUSDC'],
        "lastUpdateTimestamp": "0",
        "liquidityRate":"0",
        "borrowRate":"0",
        "liquidityCumulativeIndex":f"1{'0'*18}",
        "borrowCumulativeIndex":f"1{'0'*18}",
        "baseLTVasCollateral":"500000000000000000",
        "liquidationThreshold":"650000000000000000",
        "liquidationBonus":"100000000000000000",
        "decimals":"6",
        "borrowingEnabled": "1",
        "usageAsCollateralEnabled":"1",
        "isFreezed":"0",
        "isActive":"1"
    } 
}

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x689031a4e43df7f91ed300b83819cffdb98f7d65c18829eaa030a0614f5c6327',
 'blockHeight': 18233060,
 'blockHash': '0x6d8b7e080986ea3375bc5c3fc36d923f65353bef61a52bb03566c4cfc8b95714',
 'txIndex': 1,
 'to': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
 'stepUsed': 291320,
 'stepPrice': 12500000000,
 'cumulativeStepUsed': 291320,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Testing deposit

In [23]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(5 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x5b5f438393133ad74d87277e942b7ed3b21257f580abb2d5ff276ff911f222e7',
 'blockHeight': 18249716,
 'blockHash': '0x9572467aec9fa1492d6b19718cb8e5ca9c4f352426b46c9c9d21e27e7a7df398',
 'txIndex': 1,
 'to': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'stepUsed': 2663100,
 'stepPrice': 12500000000,
 'cumulativeStepUsed': 2663100,
 'eventLogs': [{'scoreAddress': 'cxa6b96f2396d1aab2ba66a8af0252211c1d92d757',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d',
    '0x0',
    '0x0'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx0000000000000000000000000000000000000000',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    '0x444fc77b411c5db7'],
   'data': ['0x6d696e74']},
  {'scoreAddress': 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f22

In [39]:
params ={"_reserve": contracts['sicx'], "_amount": 1 * 10 ** 18}

call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .method("borrow") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x7b07139096385f696b01e73e011536c15de41931929e405d3dfa76d8cfc38726',
 'blockHeight': 18251364,
 'blockHash': '0xeeef2583d338b4822cfac310310e40f272cc7da10d40da44f5d311be231473af',
 'txIndex': 1,
 'to': 'cx16c14999d0bf62f3875343f27c10efe4ce107623',
 'stepUsed': 4415200,
 'stepPrice': 12500000000,
 'cumulativeStepUsed': 4415200,
 'eventLogs': [{'scoreAddress': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx0bc91748db9325167f19eeef13b918c76d3768e2',
    'cx791e6684e6861ba1add73b5cf402080059ae9cee',
    '0xd3c21bcecceda1000000'],
   'data': ['0x5472616e7366657272656420746f20526577617264732053434f5245']},
  {'scoreAddress': 'cx0bc91748db9325167f19eeef13b918c76d3768e2',
   'indexed': ['Mint(int,bytes)', '0xd3c21bcecceda1000000'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx791e6684e6861ba1add73b5cf402080059ae9cee',
   'indexed': ['State(str)', 'precompute ommICX'],
   'data': []},
  {'scoreAddress': 'cxae6334850f13d

In [48]:
from checkscore.checkscore import Checkscore
from checkscore.methods import readonly

In [75]:
Checkscore(contracts['rewardDistribution']).fill_methods()

In [ ]:
## Readonly functions

In [ ]:
method = 'getAdmin'

readonly(method)

In [120]:
# params = {'_asset':}

method = 'getAssetEmission'

readonly(method)

{'cx97fd33b6002f0bb0a20f8300e2a10f89a2a390e4': '0xab54a9a6c9dc0ca', 'cx242349f8c338977dba062d728f2627831ba11cd9': '0xab54a9a6c9dc0ca', 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846': '0xab54a9a6c9dc0ca', 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a': '0xab54a9a6c9dc0ca', 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa': '0xab54a9a6c9dc0ca', 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b': '0xab54a9a6c9dc0ca'}


In [113]:
method = 'getAssets'

readonly(method)

['cx97fd33b6002f0bb0a20f8300e2a10f89a2a390e4', 'cx242349f8c338977dba062d728f2627831ba11cd9', 'cxc89e34a2c79a6082f1065b5b1ebf388009abc846', 'cxab368ceff010b0b255ba5bc22fe73cd71c53963a', 'cx8b8a9f341f9de6ceaaaa3ea9ab43c0b598ef52fa', 'cx2ec5dac634969633b7e703a0e6d1b7398e7ad62b']


In [ ]:
method = 'getDaoFund'

readonly(method)

In [ ]:
method = 'getDay'

readonly(method)

In [110]:
method = 'getDistPercentage'

readonly(method)

{'worker': '0x2c68af0bb140000', 'ommICX': '0x16345785d8a0000', 'dex': '0x16345785d8a0000', 'daoFund': '0x2c68af0bb140000'}


In [ ]:
method = 'getDistributedDay'

readonly(method)

In [ ]:
method = 'getLendingPoolDataProvider'

readonly(method)

In [ ]:
method = 'getLpToken'

readonly(method)

In [ ]:
method = 'getOmm'

readonly(method)

In [ ]:
method = 'getRecipients'

readonly(method)

In [96]:
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}

method = 'getRewardsBalance'

readonly(method, params)

{'worker': '0x0', 'ommICX': '0x0', 'dex': '0x0', 'daoFund': '0x0', 'depositBorrowRewards': '0x25e03f61372ffd6a7d6', 'ommRewards': '0x0', 'liquidityRewards': '0x0', 'total': '0x0'}


In [ ]:
method = 'getStartTimestamp'

readonly(method)

In [ ]:
method = 'getWorkerToken'

readonly(method)

In [ ]:
method = 'name'

readonly(method)

In [ ]:
params = {'_recipient': ''}

method = 'readValues'

readonly(method, params)

In [ ]:
params = {'_day': 0}

method = 'tokenDistributionPerDay'

readonly(method, params)

In [ ]:
## External Functions

In [ ]:
params = {'_amount': 0}

method = 'claimRewards'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_assetConfig': ''}

method = 'configureAssetEmission'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
method = 'distribute'
wallet = deployer_wallet

external(method, wallet)

In [ ]:
params = {'_user': '', '_userBalance': 0, '_totalSupply': 0}

method = 'handleAction'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_address': ''}

method = 'setAdmin'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_address': ''}

method = 'setDaoFund'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_ommICX': 0, '_dex': 0, '_worker': 0, '_daoFund': 0}

method = 'setDistPercentage'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_address': ''}

method = 'setLendingPoolDataProvider'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_address': ''}

method = 'setLpToken'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_address': ''}

method = 'setOmm'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_recipient': ''}

method = 'setRecipients'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_timestamp': 0}

method = 'setStartTimestamp'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_address': ''}

method = 'setWorkerToken'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
params = {'_from': '', '_value': 0, '_data': ''}

method = 'tokenFallback'
wallet = deployer_wallet

external(method, wallet, params)

In [ ]:
## Payable functions